# 04. Ленивые вычисления

<div style="text-align: right"> 
    
    Лекция 4.2 26/02/21 
    
    Кубенский Александр Александрович
    
    akoubensky@gmail.com
</div> 

## Введение в ленивые вычисления

**Опр**: *дизъюнкция по МакКарти* - безопасная дизъюнкция - сначала проверяется первый операнд, если выполняется, то второй операнд не будет выполняться.

```haskell
if x == 0 then True else y `div` x > 2
```

In [1]:
(|||)    ::  Bool -> Bool -> Bool
a ||| b  =   if a then True else b

Line 2: Redundant if
Found:
if a then True else b
Why not:
a || b

Безопасно ли выражение? Пытаясь добиться безопасности, мы повысили опасность.

```haskell
(x == 0)  |||  (y `div` x > 2)
```

Аргументы функции всегда вычисляются до того, как начинает работать функция.

Способы передачи аргументов в функцию:

- **«по значению»** – значение аргумента вычисляется и передается в функцию, перед тем как происходит вычисления
- **«по ссылке»** – аргумент – это переменная, имя которой передается в функцию (в функцию передается имя или адрес переменной)
- **«по наименованию»** – значение аргумента вычисляется при каждом обращении к нему в теле функции;

- *(!) - в функциональном программировании (!)* **«по необходимости»** – значение аргумента вычисляется при первом обращении к нему в теле функции - это будет реализована дизъюнкция по Маккарти - до вычисления *b* вычисление не дойдёт.

Если все аргументы во все процедуры передаются по
необходимости, то такой способ работы называют **ленивыми
вычислениями.**

In [2]:
f 0 2

: 

## Энергичные и ленивые вычисления

*Энергичные вычисления*: 
- аргументы всех функций вычисляются до момента входа в функцию; 
- если аргумент не может быть вычислен, то значение функции не определено (все функции – строгие по всем своим аргументам).

*Ленивые вычисления*: 

- аргументы функций не вычисляются до момента входа в функцию;
- вычисление значения аргумента происходит при первом обращении к аргументу – когда его значение нужно для выполнения примитивной функции или в момент сопоставления с образцом;
- если функция не обращается к аргументу, то его значение не вычисляется.

**Опр**: говорят, что функция *строгая по какому-нибудь из ее аргументов*, если для ее вычисления требуется обязательное вычисление этого ее аргумента.

Если функция строгая по всем своим аргументам, то говорят
просто о *строгой функции*. Например, операция логического «или» `(||)`
– строгая по первому аргументу и нестрогая по второму.

В языке Haskell все функции и конструкторы – нестрогие (ленивые) за исключением примитивных арифметических и логических операций.

In [3]:
f x y = (x == 0)  ||   (y `div` x > 2)
g x y = (x == 0)  |||  (y `div` x > 2)

«безопасны», так как и встроенная операция (||), и определенная программистом операция (|||) – ленивые.

In [4]:
f 0 10 
f 1 0 
g 0 9
g 1 0

True

False

True

False

## `Бесконечные списки`

Нестрогим является также конструктор списков (:). Конструкторы вообще все и всегда нестрогие по всем своим аргументам. Это позволяет записывать и обрабатывать такие
выражения, которые вообще никогда не могли бы быть вычислены до
конца при энергичной схеме вычислений. Такова, например, техника
обработки потенциально **бесконечных списков**.

В случае «ленивых» конструкторов аргумент может понадобиться в момент сопоставления с образцом:

In [5]:
listFrom    ::  Integer -> [Integer]
listFrom n  =   n : listFrom (n+1)

In [6]:
sumFirst   :: Integer -> [Integer] -> Integer
sumFirst 0 _       =  0
sumFirst n (e:ls)  =  e + sumFirst (n-1) ls

In [7]:
sumFirst 4 (listFrom 3)

18

Итак, оказалось, что работа с бесконечным списком окончилась
вполне успешно. Это произошло потому, что на самом деле нам *не
потребовался бесконечный список*; после вычисления первых четырех
элементов остальные не понадобились. Подобная техника может
применяться в Haskell всегда, когда это удобно.

## Значения и функции, приводящие к образованию «бесконечных» списков

In [8]:
let t1 = [2..] -- бесконечная прогрессия
let t2 = [3, 6..] -- бесконечная прогрессия с шагом 3
let t3 = [x*x | x <- [1..]] -- генератор квадратов чисел

repeat    ::  a -> [a]
repeat e  =   ls where ls = e : ls
-- repeat 5 => [5, 5, 5, 5, ...]

let t4 = repeat 5

cycle     ::  [a] -> [a]
cycle ls  =   lis where lis = ls ++ lis

-- cycle [1,2]  =>  [1,2,1,2,...]
let t5 = cycle [1, 2]


iterate      ::  (a -> a) -> a -> [a]
iterate f e  =   e : iterate f (f e)
-- iterate (+1) 1  =>  [1,2,3,...]

let t6 = iterate (+2) 1

In [9]:
sumFirst 5 t1
sumFirst 5 t2
sumFirst 5 t3
sumFirst 5 t4
sumFirst 5 t5
sumFirst 5 t6

20

45

55

25

7

25

## Функции, работающие с бесконечными списками

Большинство стандартных функций обработки списков пригодно также для работы с «бесконечными» списками. Например:

In [10]:
take 10 [1..]

[1,2,3,4,5,6,7,8,9,10]

In [11]:
sumFirst 5 (drop 10 [1..])

-- drop 10 [1..] => [11..]

65

In [12]:
[1..] !! 10

11

In [13]:
-- zipWith (*) [1..] [2..] => [2,6,12,20,...]

sumFirst 4 (zipWith (*) [1..] [2..])

40

Вычисление факториала:

In [14]:
factorials :: [Integer]
factorials = 1 : zipWith (*) [1..] factorials

In [15]:
sumFirst 6 factorials

154

Найдем последовательность, образованную разностями между последовательными квадратами натуральных чисел

In [16]:
diff      ::  (Num a) => [a] -> [a]
diff (x:l@(y:ls))  =  (y-x) : diff l

In [17]:
take 25 (diff [x*x | x <- [1..]])

[3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51]

In [18]:
take 25 (diff (diff [x*x | x <- [1..]]))

[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]

## Список простых чисел, полученный способом `решета Эратосфена`

In [19]:
primes  :: [Integer]
primes  =  sieve [2..] where
              sieve (e:ls) = e : (sieve [x | x <- ls, x `mod` e /= 0])

Line 3: Redundant bracket
Found:
e : (sieve [x | x <- ls, x `mod` e /= 0])
Why not:
e : sieve [x | x <- ls, x `mod` e /= 0]

In [20]:
pairs  ::  [Integer] -> [(Integer, Integer)]
pairs (x:l@(y:ls))  =  (x,y):(pairs l)

Line 2: Redundant bracket
Found:
(x, y) : (pairs l)
Why not:
(x, y) : pairs l

In [21]:
twins  ::  [(Integer, Integer)]
twins  =   [(x,y) | (x,y) <- pairs primes, y - x == 2]

In [22]:
take 10 twins

[(3,5),(5,7),(11,13),(17,19),(29,31),(41,43),(59,61),(71,73),(101,103),(107,109)]